In [1]:
import os

# import
import pandas as pd

In [ ]:
# import files 
main_df = pd.read_csv('../PM2.5_Vehicles_Population.csv')
df_pop = pd.read_csv('../California_Population.csv') 

In [ ]:
main_df
# main_df['Population Z Score'] = main_df['Population Z Score'].round(2)

In [ ]:
# total regestered car users/population 
def create_comparison(county):
    df = main_df[main_df['County'] == county]
    df.loc[:,'Total Cars'] = df['Number of Gas Cars'] + df['Number of EV']
    df.loc[:,'Total Cars/Population'] = (df['Total Cars'] / df['Total Population']).round(2)
    df.loc[:,'EV/Total Cars'] = (df['Number of EV'] / df['Total Cars']).round(2)
    df.loc[:,'Gas/Total Cars'] = (df['Number of Gas Cars'] / df['Total Cars']).round(2)
    return df[['Total Population Z Score','Total Cars/Population', 'EV/Total Cars', 'Gas/Total Cars']] 


In [ ]:
# so as more people got cars, more ev were regestered. 
# what is the relationship between population growth and ev adaptation?
## it seems to be positive, direct relationship 
## so as more population increases, ev car adaption also increases. i wonder if this is the same across counties? 

In [ ]:
san_diego = create_comparison('San Diego')
san_diego

In [ ]:
 # los_angeles = create_comparison('Los Angeles')
# los_angeles

In [ ]:
# orange = create_comparison('Orange')
# orange

In [ ]:
# san_fran =  create_comparison('San Francisco')
# san_fran

In [ ]:
# mono = create_comparison('Mono')
# mono 

In [ ]:
# is this crazy?
# function takes dataset from online and turns it into a merge-able df for main_df 

In [ ]:
def aged_population(dataset, datayear):
    # load files 
    df = pd.read_csv(dataset)
    df = df[df['Label (Grouping)'].str.strip() == '18 years and over']
    df = df[df['Label (Grouping)'].str.contains('18 years and over', case=False, na=False)]
    # extract unique county names
    county_names = [col.split("!!")[0] for col in df.columns if "California" in col]
    county_names = list(set(county_names))
    # extract county total and margin of error from main dataset
    county_total = [f"{county}!!Total!!Estimate" for county in county_names]
    county_margin_of_error = [f"{county}!!Total!!Margin of Error" for county in county_names]
    # making df of those
    county_total = df.loc[:, county_total]
    county_margin_of_error = df.loc[:, county_margin_of_error]
    # creating county total population estimate df 
    ct_df =county_total.transpose()
    ct_df = ct_df.reset_index() 
    # some years record >21, others >18. 
    ct_df.columns = ['County' , 'Population Estimate, >18']
    ct_df['County'] = ct_df['County'].str.split(' County').str[0]
    ct_df = ct_df.sort_values(by='County', ascending=True).reset_index(drop=True)
    
    # creating county margin of error 
    me_df = county_margin_of_error.transpose()
    me_df = me_df.reset_index() 
    me_df.columns = ['County' , 'Population Margin of Error, >18']
    me_df['County'] = me_df['County'].str.split(' County').str[0]
    me_df = me_df.sort_values(by='County', ascending=True).reset_index(drop=True) 
    
    # merging on county 
    ct_me_df = pd.merge(ct_df, me_df, how='inner', on = ['County'])
    ct_me_df['Data Year'] = datayear
    return ct_me_df

In [ ]:
# run function to create mergable df
totpop2010 = aged_population('./Pop_Over_21_2010.S0101-2024-09-27T210322.csv', 2010)
totpop2011 = aged_population('./Pop_Over_21_2011.S0101-2024-09-27T210305.csv', 2011)
totpop2012 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2012)
totpop2013 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2013)
totpop2014 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2014)
totpop2015 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2015)
totpop2016 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2016)
totpop2017 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2017)
totpop2018 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2018)
totpop2019 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2019)
totpop2020 = aged_population('./pop_over_18_2020.csv', 2020)
totpop2021 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2021)
totpop2022 = aged_population('./Pop_Over_21_2022.csv', 2022)
totpop2023 = aged_population('./Pop_Over_21_2023.S0101-2024-09-27T205616.csv', 2023)
# stacking totpop
dfs = [totpop2010, totpop2011, totpop2012, totpop2013, totpop2014, totpop2015, totpop2016, totpop2017, totpop2018, totpop2019, totpop2021, totpop2022, totpop2023]
aged_pop_2010_2023 = pd.concat(dfs, ignore_index= True)

In [ ]:
aged_pop_2010_2023['County'] = aged_pop_2010_2023['County'].str.strip()
counties = pd.DataFrame({
    'County_main_df': main_df['County'],
    'County_aged_pop_2010_2023': aged_pop_2010_2023['County']
})
counties.head()
# Get unique counties from both DataFrames
main_df_co = set(main_df['County'])
aged_pop_2010_2023_co = set(aged_pop_2010_2023['County'])

# Find differences
only_in_main_df_co = main_df_co - aged_pop_2010_2023_co
only_in_aged_pop_2010_2023_co = aged_pop_2010_2023_co - main_df_co

print("Only in main_df:", only_in_main_df_co)
print("Only in aged_pop_2010_2023:", only_in_aged_pop_2010_2023_co)

In [ ]:
# merging aged_pop_2010_2023 to main_df 
aged_pop_df = pd.merge(main_df, aged_pop_2010_2023, how='inner', on = ['Data Year', 'County'])

In [ ]:
# there is some percentages and whole numbers, let's make them all whole numbers 
def convert_to_population_estimate(row):
    # converting percentage to decimal, if applicable 
    if isinstance(row['Population Estimate, >18'], str) and '%' in row['Population Estimate, >18']:
        percent_value = float(row['Population Estimate, >18'].strip('%')) / 100
        return percent_value * row['Total Population']
    else:
        return row['Population Estimate, >18']

# apply to update 'Total Population Estimate, >21'
aged_pop_df['Population Estimate, >18'] = aged_pop_df.apply(convert_to_population_estimate, axis=1)



aged_pop_df = aged_pop_df[['County', 'Data Year', 'Total Population', 'Total Population Z Score', 
                           'Population Estimate, >18', 'Population Margin of Error, >18', 
                           'Number of Gas Cars', 'Number of EV'
                           ]]

aged_pop_df

In [ ]:
# to csv
aged_pop_df.to_csv('Main_DF_PM2.5_Vehicle_Pop.csv', index=False)

In [5]:
df_blah = pd.read_csv('population_data/Pop_Over_18_2012.csv')
df_blah

FileNotFoundError: [Errno 2] No such file or directory: 'population_data/Pop_Over_18_2012.csv'